In [5]:
import os
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

# Set your image dimensions
img_height, img_width = 90, 90
img_channels = 3  # For RGB images

# Set your dataset path within Colab
base_path = '/content/'  # Adjust this path

# Define paths for training, validation, and testing
train_folder = 'Train'
valid_folder = 'Valid'
test_folder = 'Test'

train_path = os.path.join(base_path, train_folder)
valid_path = os.path.join(base_path, valid_folder)
test_path = os.path.join(base_path, test_folder)

# Specify class names
class_names = ['Abnormal', 'Normal']

# Train data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    classes=class_names,
    subset='training'
)

# Validation data generator
val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
    valid_path,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    classes=class_names
)

# Define your custom model
custom_model = models.Sequential()
custom_model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(img_height, img_width, img_channels)))
custom_model.add(layers.MaxPooling2D((2, 2)))
custom_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
custom_model.add(layers.MaxPooling2D((2, 2)))
custom_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
custom_model.add(layers.MaxPooling2D((2, 2)))
custom_model.add(layers.Flatten())
custom_model.add(layers.Dense(512, activation='relu'))
custom_model.add(layers.Dropout(0.5))

# Load InceptionV3 (GoogLeNet) model with pre-trained weights (excluding the top layers)
inception_model = InceptionV3(input_shape=(img_height, img_width, img_channels), include_top=False, weights='imagenet')

# Fine-tune InceptionV3 layers
for layer in inception_model.layers[:249]:
    layer.trainable = False
for layer in inception_model.layers[249:]:
    layer.trainable = True

# Create an input tensor with the same shape as the InceptionV3 model's input
inception_input = layers.Input(shape=(img_height, img_width, img_channels))

# Apply InceptionV3 model to the input tensor
inception_output = inception_model(inception_input)

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
inception_output = layers.GlobalAveragePooling2D()(inception_output)

# Combine the custom model and InceptionV3 model
combined_model = models.Sequential()
combined_model.add(custom_model)
combined_model.add(layers.Dense(512, activation='relu'))
combined_model.add(layers.Dropout(0.5))
combined_model.add(layers.Dense(len(class_names), activation='softmax'))

# Compile the model
combined_model.compile(optimizer=optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = combined_model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

# Assuming you have a 'Test' folder for evaluation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical',
    classes=class_names
)

# Evaluate the model on the test set
test_loss, test_accuracy = combined_model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy}')


Found 1594 images belonging to 2 classes.
Found 1992 images belonging to 2 classes.
Epoch 1/50
100/100 [==============================] - 119s 1s/step - loss: 0.6945 - accuracy: 0.4925 - val_loss: 0.6908 - val_accuracy: 0.5070
Epoch 2/50
100/100 [==============================] - 129s 1s/step - loss: 0.6884 - accuracy: 0.5339 - val_loss: 0.6888 - val_accuracy: 0.5291
Epoch 3/50
100/100 [==============================] - 112s 1s/step - loss: 0.6906 - accuracy: 0.5307 - val_loss: 0.6869 - val_accuracy: 0.5587
Epoch 4/50
100/100 [==============================] - 117s 1s/step - loss: 0.6892 - accuracy: 0.5314 - val_loss: 0.6852 - val_accuracy: 0.5698
Epoch 5/50
100/100 [==============================] - 129s 1s/step - loss: 0.6817 - accuracy: 0.5640 - val_loss: 0.6918 - val_accuracy: 0.5126
Epoch 6/50
100/100 [==============================] - 129s 1s/step - loss: 0.6788 - accuracy: 0.5609 - val_loss: 0.6821 - val_accuracy: 0.5828
Epoch 7/50
100/100 [==============================] - 129s